#### Build zoningmods lookup table for PBA50+ and add No Project zoning values

In [1]:
import pandas as pd
import numpy as np

Build the PBA50+ zoningmods table using the parcels geography table and "zoningmodcat" column

In [2]:
# read in parcels geography
parcels_geog = pd.read_csv("M:/urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/parcels_geography_2024_02_14.csv")

In [3]:
# add a zoningmodcat column to parcels geography
parcels_geog['zoningmodcat'] = ''
for col in ["gg_id", "exd_id", "tra_id", "hra_id", "ppa_id", "ugb_id"]:
    # leave this line in only because the PBA50 BAUS code makes strings lowercase
    # and it will ensure the PBA50 strategy inputs will work
    parcels_geog[col] = parcels_geog[col].astype(str).str.lower()
    parcels_geog['zoningmodcat'] = parcels_geog['zoningmodcat'] + parcels_geog[col]

In [4]:
# build zoningmods using zoningmodcat values from parcels geography as the primary key
zoningmods = pd.DataFrame({'zoningmodcat': parcels_geog['zoningmodcat'].unique()})

# then add other zoningmods columns
for col in ["gg_id", "exd_id", "tra_id", "hra_id", "ppa_id", "ugb_id", 
            "add_bldg", "drop_bldg", "dua_up", "far_up", "dua_down", "far_down"]:
    zoningmods[col] = np.nan

zoningmods.loc[zoningmods.zoningmodcat.str.contains("gg"), "gg_id"] = "GG"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("exd"), "exd_id"] = "exd"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("tra1"), "tra_id"] = "TRA1"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("tra2"), "tra_id"] = "TRA2"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("tra3"), "tra_id"] = "TRA3"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("hra"), "hra_id"] = "HRA"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("ppa"), "ppa_id"] = "PPA"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("ugb"), "ugb_id"] = "UGB"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("uninc ugb"), "ugb_id"] = "Uninc UGB"


Add Zoning Step 1: Add unincorporated UGB zoning- Applied to No Project but not Draft Blueprint

In [5]:
# if within UGB but in unincorporated area, some upzoning is allowed for historic expansion
zoningmods.loc[zoningmods.ugb_id == 'Uninc UGB', 'dua_up'] = 1.5
zoningmods.loc[zoningmods.ugb_id == 'Uninc UGB', 'add_bldg'] = 'HS'

Step 2: Add UGB zoning, which can override unincorporated UGB

In [6]:
# and also set other "dua_up", "far_up", and "add_bldg" to nan
zoningmods.loc[zoningmods.ugb_id.isnull(), 'far_up'] = np.nan
zoningmods.loc[zoningmods.ugb_id.isnull(), 'dua_up'] = np.nan
zoningmods.loc[zoningmods.ugb_id.isnull(), 'add_bldg'] = np.nan
zoningmods.loc[zoningmods.ugb_id.isnull(), 'drop_bldg'] = np.nan
# if outside of UGB "far_down" and "dua_down" are 0
zoningmods.loc[zoningmods.ugb_id.isnull(), 'dua_down'] = int(0)
zoningmods.loc[zoningmods.ugb_id.isnull(), 'far_down'] = int(0)

In [7]:
# export
zoningmods.to_csv("M:/urban_modeling/baus/BAUS Inputs/plan_strategies/zoning_mods_PBA50Plus_NP_revised_script.csv", index=False)